# Public Key Encryption "Box"

The `box` API uses public key encryption to securely send messages between two parties who only know each others public keys.  Each party has a secret key that is used to encrypt messages.

[Libsodium Documentation](https://doc.libsodium.org/public-key_cryptography/authenticated_encryption)

In [1]:
%load_ext sql
%sql postgresql://postgres@/

Public key ncryption requires each party have a pair of keys, one public and one private, and a nonce.  The nonce doesn't have to be confidential, but it should never ever be reused with the same key. The easiest way to generate a nonce is to use `crypto_secretbox_noncegen`:

In [3]:
nonce = %sql SELECT crypto_box_noncegen::text from pgsodium.crypto_box_noncegen()
nonce = nonce[0][0]

bob = %sql SELECT public::text, secret::text FROM pgsodium.crypto_box_new_keypair()
bob_public, bob_secret = bob[0]

alice = %sql SELECT public::text, secret::text FROM pgsodium.crypto_box_new_keypair()
alice_public, alice_secret = bob[0]

 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.


## Encryption

Alice can encrypt a message to Bob with her keypair and Bob's public key.

In [12]:
box = %sql SELECT crypto_box::text FROM pgsodium.crypto_box('hello bob', :nonce, :bob_public, :alice_secret)
box = box[0][0]
print('Encrypted message from Alice to Bob is: ', box)

 * postgresql://postgres@/
1 rows affected.
Encrypted message from Alice to Bob is:  \x516414a93dc9f229a2c1d533a9c90fd82d11bf712ed12d5765


## Decryption


In [15]:
message = %sql select crypto_box_open FROM pgsodium.crypto_box_open(:box, :nonce, :alice_public, :bob_secret)
message = message[0][0]
print('Verified message is: ', message.tobytes().decode('utf8'))

 * postgresql://postgres@/
1 rows affected.
Verified message is:  hello bob
